In [1]:
from google.colab import drive
import pandas as pd
from transformers import pipeline

In [ ]:
#pip install --upgrade torch torchvision transformers

#### Applying roberta-base-sentiment for sentiment classification for the all the comments

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
all_comments = pd.read_csv("all_comments.csv")
all_comments.head()

,Unnamed: 0,post_id,Created,comment_id,body,created_utc,author_id,score,link_id,parent_id,Keyword
0,0,1jo735c,2025-03-31 16:31:21,mkpjkfy,something fantastic about a fake doctor spammi...,1.743439e+09,Gom_KBull,6755,t3_1jo735c,t3_1jo735c,Medical AI
1,1,1jo735c,2025-03-31 16:56:58,mkpomzc,https://preview.redd.it/4rh19ihv22se1.jpeg?wid...,1.743440e+09,7Hakuna_Matata7,3068,t3_1jo735c,t3_1jo735c,Medical AI
2,2,1jo735c,2025-03-31 16:49:19,mkpn47r,Board certified boob doctor.,1.743440e+09,mysteriousmeatman,607,t3_1jo735c,t3_1jo735c,Medical AI
3,3,1jo735c,2025-03-31 16:22:51,mkphvyc,This should be illegal,1.743438e+09,triple7freak1,7065,t3_1jo735c,t3_1jo735c,Medical AI
4,4,1jo735c,2025-03-31 16:35:45,mkpkfde,We’re really about to enter a grifter’s renais...,1.743439e+09,Murmido,218,t3_1jo735c,t3_1jo735c,Medical AI


In [ ]:
all_comments.shape

(20792, 11)

In [ ]:
# Load the sentiment analysis pipeline
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    device = 0,
    truncation=True,
    max_length=512
)

batch_size = 32
sentiments = []

for i in range(0, len(all_comments), batch_size):
    batch = all_comments['body'][i:i + batch_size].tolist()
    sentiments.extend(sentiment_pipeline(batch))

# Add results to DataFrame
all_comments['sentiment_label'] = [result['label'] for result in sentiments]
all_comments['sentiment_score'] = [result['score'] for result in sentiments]

# Mapping sentiment labels to numeric values (0, 1, 2)
sentiment_mapping = {
    'LABEL_0': 0,  # Negative
    'LABEL_1': 1,  # Neutral
    'LABEL_2': 2   # Positive
}

# Map the sentiment labels to your specified numeric values
all_comments['sentiment'] = all_comments['sentiment_label'].map(sentiment_mapping)

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
all_comments.head()

,Unnamed: 0,post_id,Created,comment_id,body,created_utc,author_id,score,link_id,parent_id,Keyword,sentiment_label,sentiment_score,sentiment
0,0,1jo735c,2025-03-31 16:31:21,mkpjkfy,something fantastic about a fake doctor spammi...,1.743439e+09,Gom_KBull,6755,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_0,0.505554,0
1,1,1jo735c,2025-03-31 16:56:58,mkpomzc,https://preview.redd.it/4rh19ihv22se1.jpeg?wid...,1.743440e+09,7Hakuna_Matata7,3068,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_1,0.768416,1
2,2,1jo735c,2025-03-31 16:49:19,mkpn47r,Board certified boob doctor.,1.743440e+09,mysteriousmeatman,607,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_1,0.645257,1
3,3,1jo735c,2025-03-31 16:22:51,mkphvyc,This should be illegal,1.743438e+09,triple7freak1,7065,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_0,0.900811,0
4,4,1jo735c,2025-03-31 16:35:45,mkpkfde,We’re really about to enter a grifter’s renais...,1.743439e+09,Murmido,218,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_1,0.629456,1


In [ ]:
# Randomly sample 3500 non-neutral comments for hand labelling and save in csv file

new_data = all_comments[all_comments["sentiment"] != 1] #non-neutral
sample_df = new_data.sample(n=3500, random_state=42)
sample_df.to_csv("sample1.csv")

## Training an ML model on the hand labelelled dataset

In [3]:
import pandas as pd
import string
import nltk
import joblib
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import LatentDirichletAllocation
from xgboost import XGBClassifier

#### Data processing steps

Using new sample dataset with hand labels

In [21]:
df = pd.read_csv("sample1_labelled.csv")
df = df.drop(df.columns[[0]], axis=1)
df.head()

,post_id,Created,comment_id,body,created_utc,author_id,score,link_id,parent_id,Keyword,sentiment_label,sentiment_score,sentiment,length,hand_sentiment
0,1k1dxpp,17/4/2025 22:34,mnnzrp6,It's helping me to be a better partner and a b...,1744929299,capodecina2,2,t3_1k1dxpp,t3_1k1dxpp,Medical AI,LABEL_2,0.985333,2,338,2
1,1j16q7u,1/3/2025 20:50,mfhqyr6,Excellent and level headed view. As a low key ...,1740862225,Any_Needleworker_273,15,t3_1j16q7u,t3_1j16q7u,Medical AI,LABEL_2,0.602790,2,688,2
2,1iuu7ec,21/2/2025 22:45,me2mqsb,"Fuck it, put it in charge.",1740177921,drizel,1,t3_1iuu7ec,t3_1iuu7ec,Medical AI,LABEL_0,0.898113,0,26,0
3,w5xrxe,23/7/2022 14:11,ihboknk,Who is this AL fella and why is he the answer ...,1658585460,DimSumDad72,-2,t3_w5xrxe,t3_w5xrxe,Medical AI,LABEL_0,0.779723,0,66,0
4,1cycbkq,22/5/2024 22:34,l58nrf3,No thank you! I prefer this shell.,1716417254,NaN,2,t3_1cycbkq,t3_1cycbkq,Medical AI,LABEL_2,0.675599,2,34,0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   post_id          3500 non-null   object 
 1   Created          3500 non-null   object 
 2   comment_id       3500 non-null   object 
 3   body             3500 non-null   object 
 4   created_utc      3500 non-null   int64  
 5   author_id        3332 non-null   object 
 6   score            3500 non-null   int64  
 7   link_id          3500 non-null   object 
 8   parent_id        3500 non-null   object 
 9   Keyword          3500 non-null   object 
 10  sentiment_label  3500 non-null   object 
 11  sentiment_score  3500 non-null   float64
 12  sentiment        3500 non-null   int64  
 13  length           3500 non-null   int64  
 14  hand_sentiment   3491 non-null   object 
dtypes: float64(1), int64(4), object(10)
memory usage: 410.3+ KB


In [24]:
labelled_df = df.iloc[:3500]
null_rows = labelled_df[labelled_df["hand_sentiment"].isna()]
null_rows

#Some hand label values are missing, will fill them with values from pre-trained model

,post_id,Created,comment_id,body,created_utc,author_id,score,link_id,parent_id,Keyword,sentiment_label,sentiment_score,sentiment,length,hand_sentiment
30,10v7j59,6/2/2023 16:59,j7ggf8x,I can’t see a list 😣 why?,1675702776,Rasch3n,2,t3_10v7j59,t3_10v7j59,Trust in AI diagnostics,LABEL_0,0.875403,0,25,NaN
191,1gpmakh,16/11/2024 8:32,lxelulk,Open Evidence has been lifechanging! I can't ...,1731745962,HonestScholar822,2,t3_1gpmakh,t3_1gpmakh,AI in medicine,LABEL_2,0.699919,2,170,NaN
235,1ilrar2,10/2/2025 1:25,mbxxekf,my pleasure; I think its always etter to be cu...,1739150723,NaxusNox,2,t3_1ilrar2,t1_mbxqou1,AI in medicine,LABEL_2,0.866704,2,387,NaN
398,1i69dwr,23/1/2025 23:52,m8tjijx,"I used to live in Fort Mojave, AZ. There is a ...",1737676334,Morphecto_Solrac,1,t3_1i69dwr,t3_1i69dwr,Medical AI,LABEL_0,0.858986,0,204,NaN
402,18b29xf,5/12/2023 4:01,kc1x9t3,"I’m so sorry, I wish I could give you any advi...",1701748867,7MrKai,39,t3_18b29xf,t3_18b29xf,Medical AI,LABEL_0,0.913129,0,364,NaN
1521,wje41f,8/8/2022 19:32,ijhd1dg,Finally a good patch this year. \n\nFinally th...,1659987144,ReGGgas,22,t3_wje41f,t3_wje41f,Medical AI,LABEL_2,0.737726,2,484,NaN
1522,145c91f,9/6/2023 23:34,jnlfz4h,This is one of the most heartbreaking things I...,1686353648,LoRdVNestEd,6,t3_145c91f,t3_145c91f,Medical AI,LABEL_0,0.964150,0,175,NaN
1523,1k1dxpp,17/4/2025 18:02,mnmi05e,I agree. I had to stop therapy for financial r...,1744912925,Guilty-Chocolate-597,7,t3_1k1dxpp,t3_1k1dxpp,Medical AI,LABEL_2,0.649986,2,805,NaN
1524,1g9iobo,22/10/2024 16:32,lt71o0w,I do not understand why so many blame economic...,1729614726,NaN,-8,t3_1g9iobo,t3_1g9iobo,Medical AI,LABEL_0,0.676658,0,905,NaN


In [25]:
#Fill null values from the pre-trained results
labelled_df.loc[:, 'hand_sentiment'] = labelled_df['hand_sentiment'].fillna(labelled_df['sentiment'])

labelled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   post_id          3500 non-null   object 
 1   Created          3500 non-null   object 
 2   comment_id       3500 non-null   object 
 3   body             3500 non-null   object 
 4   created_utc      3500 non-null   int64  
 5   author_id        3332 non-null   object 
 6   score            3500 non-null   int64  
 7   link_id          3500 non-null   object 
 8   parent_id        3500 non-null   object 
 9   Keyword          3500 non-null   object 
 10  sentiment_label  3500 non-null   object 
 11  sentiment_score  3500 non-null   float64
 12  sentiment        3500 non-null   int64  
 13  length           3500 non-null   int64  
 14  hand_sentiment   3500 non-null   object 
dtypes: float64(1), int64(4), object(10)
memory usage: 410.3+ KB


In [26]:
labelled_df["hand_sentiment"].isna().sum()

np.int64(0)

In [29]:
# Making the data consistent
# Need to run this cell 2 times
labelled_df['hand_sentiment'] = labelled_df['hand_sentiment'].replace(
    {0.0: 0, 2.0: 1, 2 : 1, 'o': 0, '-': 0, ' ': 0}
)

labelled_df.loc[:, 'hand_sentiment'] = labelled_df['hand_sentiment'].astype(int)

print(labelled_df['hand_sentiment'].value_counts())

hand_sentiment
0    2258
1    1242
Name: count, dtype: int64


<ipython-input-29-1ff53c29b7ac>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labelled_df['hand_sentiment'] = labelled_df['hand_sentiment'].replace(


In [ ]:
# Calculate value counts and percentages
value_counts = labelled_df['hand_sentiment'].value_counts(normalize=True) * 100

print(value_counts)

In [30]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [31]:
# Text preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)

    # Removing stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Rejoining tokens
    cleaned_text = ' '.join(tokens)
    return cleaned_text

# Apply preprocessing
labelled_df.loc[:, 'cleaned_text'] = labelled_df['body'].apply(preprocess_text)

In [32]:
labelled_df.shape

(3500, 16)

### Model training

In [33]:
#Data splitting

X = labelled_df['cleaned_text']
y = labelled_df['hand_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Multinomial Naive Bayes": MultinomialNB(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

vectorizers = {
    "Count Vectorizer": CountVectorizer(max_features=5000),
    "TF-IDF Vectorizer": TfidfVectorizer(max_features=5000)
}

# Function to train and evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test, vectorizer, use_lda = False):
    # Apply the vectorizer
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    # If using LDA, fit the model separately
    if use_lda == False:
        # Fit the model
        model.fit(X_train_vec, y_train)
        y_pred = model.predict(X_test_vec)

        # Print results
        print(f"{model.__class__.__name__} with {vectorizer.__class__.__name__} Classification Report:")
        print(classification_report(y_test, y_pred))
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("\n")

    # If using LDA, transform the data
    else:

        lda = LatentDirichletAllocation(n_components=5, random_state=42)
        X_train_lda = lda.fit_transform(X_train_vec)
        X_test_lda = lda.transform(X_test_vec)

        model.fit(X_train_lda, y_train)
        y_pred = model.predict(X_test_lda)

        print(f"{model.__class__.__name__} with LDA Classification Report:")
        print(classification_report(y_test, y_pred))
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("\n")

In [35]:
# Evaluate each model with Count Vectorizer and TF-IDF
for model_name, model in models.items():
    for vectorizer_name, vectorizer in vectorizers.items():
        evaluate_model(model, X_train, X_test, y_train, y_test, vectorizer)

LogisticRegression with CountVectorizer Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       441
           1       0.75      0.66      0.70       259

    accuracy                           0.79       700
   macro avg       0.78      0.76      0.77       700
weighted avg       0.79      0.79      0.79       700

Accuracy: 0.7928571428571428


LogisticRegression with TfidfVectorizer Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.99      0.87       441
           1       0.96      0.51      0.67       259

    accuracy                           0.81       700
   macro avg       0.87      0.75      0.77       700
weighted avg       0.84      0.81      0.79       700

Accuracy: 0.8114285714285714


SVC with CountVectorizer Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.97      0.85       441
 

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:57:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier with CountVectorizer Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.93      0.84       441
           1       0.81      0.54      0.65       259

    accuracy                           0.78       700
   macro avg       0.79      0.73      0.75       700
weighted avg       0.79      0.78      0.77       700

Accuracy: 0.7842857142857143




/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:57:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier with TfidfVectorizer Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.92      0.85       441
           1       0.81      0.58      0.67       259

    accuracy                           0.79       700
   macro avg       0.80      0.75      0.76       700
weighted avg       0.79      0.79      0.78       700

Accuracy: 0.7914285714285715




In [36]:
# Evaluate each model with LDA
for model_name, model in models.items():
    count_vectorizer = CountVectorizer(max_features=5000)
    evaluate_model(model, X_train, X_test, y_train, y_test, count_vectorizer, use_lda=True)

LogisticRegression with LDA Classification Report:
              precision    recall  f1-score   support

           0       0.63      1.00      0.77       441
           1       0.00      0.00      0.00       259

    accuracy                           0.63       700
   macro avg       0.32      0.50      0.39       700
weighted avg       0.40      0.63      0.49       700

Accuracy: 0.63




/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SVC with LDA Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.96      0.78       441
           1       0.70      0.16      0.26       259

    accuracy                           0.66       700
   macro avg       0.68      0.56      0.52       700
weighted avg       0.68      0.66      0.59       700

Accuracy: 0.6642857142857143


RandomForestClassifier with LDA Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.78      0.73       441
           1       0.50      0.38      0.43       259

    accuracy                           0.63       700
   macro avg       0.59      0.58      0.58       700
weighted avg       0.61      0.63      0.62       700

Accuracy: 0.63


MultinomialNB with LDA Classification Report:
              precision    recall  f1-score   support

           0       0.63      1.00      0.77       441
           1       0.00      0.00      0.00       25

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XGBClassifier with LDA Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.79      0.73       441
           1       0.52      0.39      0.44       259

    accuracy                           0.64       700
   macro avg       0.60      0.59      0.59       700
weighted avg       0.62      0.64      0.63       700

Accuracy: 0.6385714285714286




/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:58:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


#### Performing hyperparameter tuning for the 2 best models (LR + TFIDF, MNB + TF) using Grid Search CV

In [38]:
vectorizer_tf = CountVectorizer(max_features=5000)
X_train_tf = vectorizer_tf.fit_transform(X_train)
X_test_tf = vectorizer_tf.transform(X_test)

vectorizer_tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

In [39]:
logistic_param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}

# Grid Search for Logistic Regression with TF-IDF
logistic_grid_search = GridSearchCV(LogisticRegression(max_iter=1000),
                                     logistic_param_grid,
                                     scoring='accuracy',
                                     cv=5)
logistic_grid_search.fit(X_train_tfidf, y_train)

# Best Logistic Regression Model
best_logistic_model = logistic_grid_search.best_estimator_
y_pred_logistic = best_logistic_model.predict(X_test_tf)

# Evaluation
print("Best Logistic Regression Parameters:", logistic_grid_search.best_params_)
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_logistic))
print(accuracy_score(y_test, y_pred_logistic))

Best Logistic Regression Parameters: {'C': 10, 'solver': 'liblinear'}
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.79      0.82       441
           1       0.68      0.76      0.72       259

    accuracy                           0.78       700
   macro avg       0.76      0.77      0.77       700
weighted avg       0.79      0.78      0.78       700

0.7785714285714286


In [40]:
# Grid Search for Multinomial Naive Bayes with TF

nb_param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'fit_prior': [True, False]
}

nb_grid_search = GridSearchCV(MultinomialNB(),
                               nb_param_grid,
                               scoring='accuracy',
                               cv=5)
nb_grid_search.fit(X_train_tf, y_train)

# Best Naive Bayes Model
best_nb_model = nb_grid_search.best_estimator_
y_pred_nb = best_nb_model.predict(X_test_tfidf)

print("Best Multinomial Naive Bayes Parameters:", nb_grid_search.best_params_)
print("Multinomial Naive Bayes Classification Report:")
print(classification_report(y_test, y_pred_nb))
print(accuracy_score(y_test, y_pred_nb))

Best Multinomial Naive Bayes Parameters: {'alpha': 0.1, 'fit_prior': True}
Multinomial Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.94      0.85       441
           1       0.84      0.55      0.67       259

    accuracy                           0.80       700
   macro avg       0.81      0.75      0.76       700
weighted avg       0.80      0.80      0.78       700

0.7957142857142857


In [41]:
# Grid Search for Random Forest with TF-IDF

rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid Search for Random Forest
rf_grid_search = GridSearchCV(RandomForestClassifier(),
                               rf_param_grid,
                               scoring='accuracy',
                               cv=5)
rf_grid_search.fit(X_train_tfidf, y_train)

# Best Random Forest Model
best_rf_model = rf_grid_search.best_estimator_
y_pred_rf = best_rf_model.predict(X_test_tfidf)

# Evaluation
print("Best Random Forest Parameters:", rf_grid_search.best_params_)
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

Best Random Forest Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.91      0.84       441
           1       0.79      0.58      0.67       259

    accuracy                           0.79       700
   macro avg       0.79      0.74      0.75       700
weighted avg       0.79      0.79      0.78       700



#### Applying the best model to all the comments

In [42]:
# Train Multinomial Naive Bayes model with default parameters
nb_model = MultinomialNB()
nb_model.fit(X_train_tf, y_train)

nb_pred = nb_model.predict(X_test_tf)
print("Training Accuracy:", accuracy_score(y_test, nb_pred))

Training Accuracy: 0.8214285714285714


In [45]:
# Transform the new data using the tf vectorizer
all_comments['cleaned_text'] = all_comments['body'].apply(preprocess_text)
X_all_comments_tf = vectorizer_tf.transform(all_comments['cleaned_text'])

# Predict using the trained model
predictions = nb_model.predict(X_all_comments_tf)

# Add mysentiment to all_comments
all_comments['mysentiment'] = predictions

In [46]:
all_comments.head()

,Unnamed: 0,post_id,Created,comment_id,body,created_utc,author_id,score,link_id,parent_id,Keyword,cleaned_text,mysentiment
0,0,1jo735c,2025-03-31 16:31:21,mkpjkfy,something fantastic about a fake doctor spammi...,1.743439e+09,Gom_KBull,6755,t3_1jo735c,t3_1jo735c,Medical AI,something fantastic fake doctor spamming b00b,0
1,1,1jo735c,2025-03-31 16:56:58,mkpomzc,https://preview.redd.it/4rh19ihv22se1.jpeg?wid...,1.743440e+09,7Hakuna_Matata7,3068,t3_1jo735c,t3_1jo735c,Medical AI,httpspreviewreddit4rh19ihv22se1jpegwidth1026fo...,0
2,2,1jo735c,2025-03-31 16:49:19,mkpn47r,Board certified boob doctor.,1.743440e+09,mysteriousmeatman,607,t3_1jo735c,t3_1jo735c,Medical AI,board certified boob doctor,0
3,3,1jo735c,2025-03-31 16:22:51,mkphvyc,This should be illegal,1.743438e+09,triple7freak1,7065,t3_1jo735c,t3_1jo735c,Medical AI,illegal,0
4,4,1jo735c,2025-03-31 16:35:45,mkpkfde,We’re really about to enter a grifter’s renais...,1.743439e+09,Murmido,218,t3_1jo735c,t3_1jo735c,Medical AI,’ really enter grifter ’ renaissance ’,0


In [47]:
# Save the final labelled version
all_comments.to_csv("all_comments_labelled.csv")